# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Aqueduct-data-3.0" data-toc-modified-id="Aqueduct-data-3.0-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Aqueduct data 3.0</a></div><div class="lev2 toc-item"><a href="#Data-status" data-toc-modified-id="Data-status-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span><a href="https://docs.google.com/spreadsheets/d/1U7He0q8KBcOuZBNO_65XuUo_1TxCYVGYiuylESOoIPk/edit" target="_blank">Data status</a></a></div><div class="lev2 toc-item"><a href="#Data-exploration:" data-toc-modified-id="Data-exploration:-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Data exploration:</a></div><div class="lev2 toc-item"><a href="#Zonal-statistics,-generate-subasin-level-stats" data-toc-modified-id="Zonal-statistics,-generate-subasin-level-stats-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Zonal statistics, generate subasin level stats</a></div><div class="lev2 toc-item"><a href="#Baseline-Water-stress-reconfiguration" data-toc-modified-id="Baseline-Water-stress-reconfiguration-14"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Baseline Water stress reconfiguration</a></div><div class="lev2 toc-item"><a href="#GroundWater" data-toc-modified-id="GroundWater-15"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>GroundWater</a></div><div class="lev2 toc-item"><a href="#Arid-and-low-water-use-basins" data-toc-modified-id="Arid-and-low-water-use-basins-16"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Arid and low water use basins</a></div><div class="lev2 toc-item"><a href="#Merge-indicators-tables-into-one-big-table" data-toc-modified-id="Merge-indicators-tables-into-one-big-table-17"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Merge indicators tables into one big table</a></div><div class="lev2 toc-item"><a href="#Categorization-computation" data-toc-modified-id="Categorization-computation-18"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Categorization computation</a></div><div class="lev2 toc-item"><a href="#Raster-reclassification" data-toc-modified-id="Raster-reclassification-19"><span class="toc-item-num">1.9&nbsp;&nbsp;</span>Raster reclassification</a></div><div class="lev2 toc-item"><a href="#Rankings" data-toc-modified-id="Rankings-110"><span class="toc-item-num">1.10&nbsp;&nbsp;</span>Rankings</a></div><div class="lev2 toc-item"><a href="#Data-Migration-from-wri-01-account-to-wri-rw" data-toc-modified-id="Data-Migration-from-wri-01-account-to-wri-rw-111"><span class="toc-item-num">1.11&nbsp;&nbsp;</span>Data Migration from wri-01 account to wri-rw</a></div>

# Aqueduct data 3.0

Todo List: 
- [X] Explore data 
- [X] Zonal statisticts at basin level for the raster datasets
- [X] Convert everything to our current format
- [X] Add categorization and export table as csv
- [X] Rasters visual reclasification 0-5 
- [x] Arid - low water use classification
- [X] Ground water
- [X] Migrate data to RW cartoData account.


Maybe rework to obtain 2 tables one for the platform and the other one with the data?
    - 

In [31]:

%matplotlib inline
import __future__
from IPython.display import display, Markdown
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import pandas as pd
import numpy
import geopandas as gpd
import rasterio
import fiona
from rasterio.mask import mask
from rasterio.plot import show
from rasterstats import zonal_stats
from math import log, ceil, floor
import os

In [32]:
base_path=os.getcwd().split('/work')[0]
data_path='/work/data/aqueduct/data_source/aqueduct/v3/toAlicia/calculatedWS03.csv'

## [Data status](https://docs.google.com/spreadsheets/d/1U7He0q8KBcOuZBNO_65XuUo_1TxCYVGYiuylESOoIPk/edit)

<img  src="HL5Y510A14UMVLTW6EINVGT83ATDHN25.png"/>

## Data exploration:

##### Datasets:

Dataset list currently available
###### Tables:

* baseline water stress monthly and yearly [dimensionless]
    * calculatedWS03.csv: Contains the next data attached to it
* Ground water stress [dimensionless m^3/m^3 or m/m]
    * aquifer_table_sorted.txt: Contains:
* Groundwater table Declining Trend [cm/year]
    * aquifer_table_sorted.txt: Contains  

###### rasters:

* drought severity [dimensionless]:
    * global_droughtseveritystandardisedsoilmoisture_5min_1960-2014.tif
    * global_droughtseveritystandardisedstreamflow_5min_1960-2014.tif
* Interannual variability [dimensionless]: 
    * global_interannualvariabilitywatersupply_5min_1960-2014.tif
* Seasonal variability [dimensionless]: 
    * global_q1seasonalvariabilitywatersupply_5min_1960-2014.tif
    * global_q2seasonalvariabilitywatersupply_5min_1960-2014.tif
    * global_q3seasonalvariabilitywatersupply_5min_1960-2014.tif
    * global_q4seasonalvariabilitywatersupply_5min_1960-2014.tif  
    
###### Geometries:
* **hybas_merged_custom_level6_V01.shp**: contains the geometries of HYDRO BASIN level 6.
* **unionADM01Hybas6V02WHYMAPV05.shp**: Merge between aquifers, admin levels 0,1 and HYDRO BASIN level 6.


### Baseline water stress, Yearly and monthly

In [44]:
inData = pd.read_csv(base_path+data_path)

In [ ]:
print(inData.shape)
display(inData.head(2))

In [ ]:
print(inData.columns.values)

* Columns to be keept for the data visualization and to be transform into indicators table:   

In [33]:
column_list = ['PFAF_ID','ws_yearY2014']
for month in list(range(1,13)):
    column_list.append("ws_monthY2014M%02d" % (month))
column_list

['PFAF_ID',
 'ws_yearY2014',
 'ws_monthY2014M01',
 'ws_monthY2014M02',
 'ws_monthY2014M03',
 'ws_monthY2014M04',
 'ws_monthY2014M05',
 'ws_monthY2014M06',
 'ws_monthY2014M07',
 'ws_monthY2014M08',
 'ws_monthY2014M09',
 'ws_monthY2014M10',
 'ws_monthY2014M11',
 'ws_monthY2014M12']

* Columns to use as a Arid and low water use

In [34]:
col_list = ['PFAF_ID','local_sum_volumem3_TotWW_yearY2014','local_sum_volumem3_Runoff_yearY2014', 'upstream_sum_volumem3runoff_annua','upstream_sum_volumem3_TotWN_yearY2014', 'meanarea30sm2', 'countarea30sm2']
col_list

['PFAF_ID',
 'local_sum_volumem3_TotWW_yearY2014',
 'local_sum_volumem3_Runoff_yearY2014',
 'upstream_sum_volumem3runoff_annua',
 'upstream_sum_volumem3_TotWN_yearY2014',
 'meanarea30sm2',
 'countarea30sm2']

### Ground water stress

In [35]:
groundData = pd.read_csv(base_path+'/work/data/aqueduct/data_source/aqueduct/v3/toAlicia/ground_water/aquifer_table_sorted.txt',sep=';')

display(groundData.head())

grwcolumns=['AqID_spatial_unit','groundwater_stress','slope_of_decline_cm.year-1']

,AqID_spatial_unit,groundwater_stress,slope_of_decline_cm.year-1,p_value,r_squared
0,0,-9999.0,-9999.0,-9999.0,-9999.0
1,1,-9999.0,-9999.0,-9999.0,-9999.0
2,2,-9999.0,-9999.0,-9999.0,-9999.0
3,3,-9999.0,-9999.0,-9999.0,-9999.0
4,5,-9999.0,-9999.0,-9999.0,-9999.0


### Geometries

#### Basins intersected with aquifers and two administrative levels
only for visual purpouses in the platform

In [ ]:
geoVisualData = gpd.read_file(base_path+'/work/data/aqueduct/data_source/aqueduct/v3/toAlicia/shape/unionADM01Hybas6V02WHYMAPV05.shp')

display(geoVisualData.head(2))

#### Basins geometries

In [36]:
geoInData = gpd.read_file(base_path+'/work/data/aqueduct/data_source/aqueduct/v3/toAlicia/shape/hybas_merged_custom_level6_V01.shp')

display(geoInData.head(2))

,COAST,DIST_MAIN,DIST_SINK,ENDO,HYBAS_ID,HYBAS_IDIn,MAIN_BAS,NEXT_DOWN,NEXT_DOWNI,NEXT_SINK,ORDER_,PFAF_ID,SORT,SUB_AREA,UP_AREA,geometry
0,1,0.0,0.0,0,1.060000e+09,1.060000e+09,1.060000e+09,0.0,0.0,1.060000e+09,0,111011,1.0,1890.8,1890.8,"POLYGON ((32.36666666666668 29.6791666666667, ..."
1,0,0.0,0.0,0,1.060000e+09,1.060000e+09,1.060000e+09,0.0,0.0,1.060000e+09,1,111012,2.0,2925.9,2925.9,"POLYGON ((31.73333333333336 29.88333333333337,..."


In [ ]:
#geoInData.plot()

In [ ]:
for column in geoInData.columns.values:
    print(column)

Keep all columns; 

### Pixel level datasets: 

* drought severity:
    * soil moisture
    * streamflow
* interannual variability:
* seasonal variability:
    * q1
    * q2
    * q3
    * q4

#### Generator

In [38]:
"""
Tiff config object:
[
{'dataset': , 'indicator': , 
'period': , 'period_value': , 
'year': 2014, 
'data_type': 'absolute', 
'scenario': 'historic'}
]
"""
baseLocation = base_path+'/work/data/aqueduct/data_source/aqueduct/v3/pixelIndicatorsAlicia/' 

tifss= [{'dataset': "global_droughtseveritystandardisedsoilmoisture_5min_1960-2014.tif", 'indicator': 'drought_severity_soil', 'period': 'year', 'period_value': None, 'year': 2014, 'data_type': 'absolute', 'scenario': 'historic'},
        {'dataset': "global_droughtseveritystandardisedstreamflow_5min_1960-2014.tif", 'indicator': 'drought_severity_flow', 'period': 'year', 'period_value': None, 'year': 2014, 'data_type': 'absolute', 'scenario': 'historic'},
        {'dataset': "global_interannualvariabilitywatersupply_5min_1960-2014.tif", 'indicator': 'interannual_variability', 'period': 'year', 'period_value': None, 'year': 2014, 'data_type': 'absolute', 'scenario': 'historic'},
        {'dataset': "global_q1seasonalvariabilitywatersupply_5min_1960-2014.tif", 'indicator': 'seasonal_variability', 'period': 'quarter', 'period_value': 1, 'year': 2014, 'data_type': 'absolute', 'scenario': 'historic'},
        {'dataset': "global_q2seasonalvariabilitywatersupply_5min_1960-2014.tif", 'indicator': 'seasonal_variability', 'period': 'quarter', 'period_value': 2, 'year': 2014, 'data_type': 'absolute', 'scenario': 'historic'},
        {'dataset': "global_q3seasonalvariabilitywatersupply_5min_1960-2014.tif", 'indicator': 'seasonal_variability', 'period': 'quarter', 'period_value': 3, 'year': 2014, 'data_type': 'absolute', 'scenario': 'historic'},
        {'dataset': "global_q4seasonalvariabilitywatersupply_5min_1960-2014.tif", 'indicator': 'seasonal_variability', 'period': 'quarter', 'period_value': 4, 'year': 2014, 'data_type': 'absolute', 'scenario': 'historic'}
] 

#tifss= [{'dataset': "global_interannualvariabilitywatersupply_5min_1960-2014.tif", 'indicator': 'interannual_variability', 'period': 'year', 'period_value': None, 'year': 2014, 'data_type': 'absolute', 'scenario': 'historic'},
 #       ] 

In [ ]:
n = 1
for image in tifss:
    with rasterio.open(baseLocation+image['dataset']) as tif:
        display(Markdown('### '+image['dataset'].split('.')[0]))
        print(tif.profile)
        rasterData=tif.read(1, masked=True)
        print(rasterData)
        profile = tif.profile
        f1 = plt.figure(n)
        plt.imshow(rasterData[:,:],  cmap='pink')
        f1.show()
        f2= plt.figure(n+1)
        plt.hist(rasterData.compressed().ravel(),bins=[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.5,2])
        f2.show()
    n=n+2
    
    

## Zonal statistics, generate subasin level stats

What's are the stats? The mean  


In [39]:
def zonal_statistics(baseLocation, geomDatasets, cfgTif):
    """
    This function generates the zonal statistics (mean) between the rasters and the basins.
    """
    dataset=[]
    with rasterio.open(baseLocation + cfgTif['dataset']) as tif:
        rasterData=tif.read(1)
        profile = tif.profile
    for index, row in geoInData.iterrows():
        mean = zonal_stats(row['geometry'], rasterData, stats=['mean'], band=1, all_touched=True,raster_out=False, affine=profile['transform'], nodata=profile['nodata'])
        dataset.append({'indicator': cfgTif['indicator'], 
                        'value': mean[0]['mean'], 
                        'basinid':row['PFAF_ID'],
                        'aqid':None,
                        'period': cfgTif['period'], 
                        'period_value': cfgTif['period_value'], 
                        'year': cfgTif['year'], 
                        'data_type': cfgTif['data_type'], 
                        'scenario': cfgTif['scenario']
                       })
    return pd.DataFrame(dataset, columns=dataset[0].keys())




## Baseline Water stress reconfiguration

In [40]:
def unpivot(frame):
    """
    This function unpivot water indicators selected (converts columns into rows in an ordered way).
    """
    mapK={
        'ws':'water_stess'
    }
    N, K = frame.shape
    indicator = [mapK[i.split('_')[0]] for  i in frame.columns]
    period = [i.split('_')[1].split('Y')[0] for  i in frame.columns]
    period_value = [int(i.split('_')[1].split('Y')[1].split('M')[1]) if len(i.split('_')[1].split('Y')[1].split('M')) > 1 else None for  i in frame.columns]
    year= [int(i.split('_')[1].split('Y')[1].split('M')[0]) for  i in frame.columns]

    
    data = {'value' : frame.values.ravel('F'),
            'indicator' : numpy.array(indicator).repeat(N),
            'aqid': None,
            'basinid' : numpy.tile(numpy.asarray(frame.index), K),
            'period': numpy.array(period).repeat(N),
            'period_value': numpy.asarray(period_value).repeat(N),
            'year': numpy.array(year).repeat(N),
            'data_type': 'absolute',
            'scenario': 'historic'
           }
    return pd.DataFrame(data, columns=['basinid','aqid','indicator', 'period','period_value','year','data_type','scenario','value'])


## GroundWater

In [41]:
def GroundWaterData(frame):
    """
    This function unpivot Groundwater indicators selected (converts columns into rows in an ordered way).
    """
    mapK={
        'groundwater_stress':'groundwater_stress',
        'slope_of_decline_cm.year-1':'groundwater_decline_trend'
        
    }
    N, K = frame.shape
    indicator = [mapK[i] for  i in frame.columns]
    
    data = {'value' : frame.values.ravel('F'),
            'indicator' : numpy.array(indicator).repeat(N),
            'aqid': numpy.tile(numpy.asarray(frame.index), K),
            'basinid' : None,
            'period': 'year',
            'period_value': None,
            'year': 2014,
            'data_type': 'absolute',
            'scenario': 'historic'
           }
    return pd.DataFrame(data, columns=['basinid', 'aqid','indicator', 'period','period_value','year','data_type','scenario','value'])


## Arid and low water use basins

Calculous information provided in the [following notebook](http://nbviewer.jupyter.org/gist/rutgerhofste/f38debeea3bd6106b7acdc2c77ecf69e).  


* Arid : Available Blue Water** : $$ A\color{blue}BWater  <0.03 m/(m^2*year)$$  
* Low water use:  $$waterWithdrawal(WW) < 0.012 m/(m^2*year) $$

*A basin needs to satisfy both (AND) conditions to be masked as Arid AND low water use.*  
\*\* **Available blue water calculation:** $$ A\color{blue}BWater = upstreamRunoff – upstreamConsumption(WN) + localRunoff $$
 

In [42]:
def Blue_water(x):
    '''
    Will generate Available blue water column 
    '''  
    return (x['upstream_sum_volumem3runoff_annua']-x['upstream_sum_volumem3_TotWN_yearY2014']+x['local_sum_volumem3_Runoff_yearY2014'])/x['area_m2']

def arLoWa(x):
    '''
    Will generate a boolean column that will indicate if that geometry is arid or not 
    '''  
    if (x['local_sum_volumem3_TotWW_yearY2014']/x['area_m2'])<0.012 and x['ABWater']<0.03:
        return True
    else:
        return False

def arid(dataframe):
    '''
    Will return f dataframe with ['PFAF_ID', 'is_AridLoW'] columns
    '''
    dataframe['area_m2']= dataframe['meanarea30sm2']*dataframe['countarea30sm2']
    dataframe['ABWater']=dataframe.apply(Blue_water, axis=1)
    dataframe['is_AridLoW']=dataframe.apply(arLoWa, axis=1)
    
    return dataframe[['PFAF_ID','ABWater', 'is_AridLoW']] 

In [45]:
wv_arid = arid(inData[col_list])
display(wv_arid.tail(5))
display(wv_arid.groupby('is_AridLoW')['is_AridLoW'].count())
arid_list=wv_arid.loc[wv_arid['is_AridLoW']==True]['PFAF_ID'].values

/Users/alicia/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/alicia/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/alicia/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-do

,PFAF_ID,ABWater,is_AridLoW
16392,312042,0.081694,False
16393,312016,0.076486,False
16394,312804,0.012636,True
16395,322660,0.052839,False
16396,312670,0.103450,False


is_AridLoW
False    13383
True      3014
Name: is_AridLoW, dtype: int64

### Negative Available Blue water
in the dat from Utrecht University it is possible to have negative local runoff values, leading to a negative available blue water value. These areas are water stressed and should have a categroy 5. This will affect 278 basins that have negative water, 486 basins with 0 water availabel and hence 764 basins in total (<=0)

In [46]:
stresed_list=[]

stresed_list = wv_arid.loc[wv_arid['ABWater']<= 0]['PFAF_ID'].values



## Merge indicators tables into one big table

In [47]:
outData=[]
for dataset in tifss:
    outData.append(zonal_statistics(baseLocation, geoInData, dataset))
outData.append(unpivot(inData[column_list].set_index(['PFAF_ID'])))
outData.append(GroundWaterData(groundData[grwcolumns].set_index(['AqID_spatial_unit'])))
f = pd.concat(outData)
print(f.shape)

(334510, 9)


## Categorization computation 

[Threshold functions](https://github.com/rutgerhofste/Aqueduct30Docker/tree/master/notebooks/thresholds)



* **Baseline water stress:** $$f(x) = \frac{{LN(x)-LN(0.1)}}{LN(2) } + 1$$
    - 0.1 represents minimum interest value  
    - 2 represent the maximum interest value  
    ##### Arid & low water use:

    * Arid : Available Blue Water** : $$ A\color{blue}BWater  <0.03 m/(m^2*year)$$  
    * Low water use:  $$waterWithdrawal(WW) < 0.012 m/(m^2*year) $$

    *A basin needs to satisfy both (AND) conditions to be masked as Arid AND low water use.*  
    \*\* **Available blue water calculation:** $$ A\color{blue}BWater = upstreamRunoff – upstreamConsumption(WN) + localRunoff $$
 
* **Drought severity:** $$f(x) = 5\frac{{x-min}}{max-min} + 1$$
* **Interannual variability:** $$f(x) = 5\frac{{x-min}}{max-min} + 1$$
* **Seasonal variability:** $$f(x) = 5\frac{{x-min}}{max-min} + 1$$

* **Groundwater stress:** $$f(x)=\begin{cases}null & x = -9999\\ 1 & -9999>x< 1\\2 & 1>x< 5\\3 & 5 >x < 10\\4 & 10>x\leq20\\5 & x\geq20\end{cases}$$
* **Groundwater Declining Trend:** $$f(x)=\begin{cases}null & x = -9998\\ 1 & x< 0\\2 & 0>x< 2\\3 & 2 >x < 8\\4 & x\geq8\end{cases}$$

*where x is the raw value*


In [48]:
f.head(8)

,aqid,basinid,data_type,indicator,period,period_value,scenario,value,year
0,None,111011,absolute,drought_severity_soil,year,None,historic,0.442394,2014
1,None,111012,absolute,drought_severity_soil,year,None,historic,0.224826,2014
2,None,111013,absolute,drought_severity_soil,year,None,historic,0.204518,2014
3,None,111014,absolute,drought_severity_soil,year,None,historic,0.243267,2014
4,None,111015,absolute,drought_severity_soil,year,None,historic,0.181155,2014
5,None,111016,absolute,drought_severity_soil,year,None,historic,0.421237,2014
6,None,111017,absolute,drought_severity_soil,year,None,historic,0.176668,2014
7,None,111018,absolute,drought_severity_soil,year,None,historic,0.165748,2014


In [49]:
def indicatorStc(frame):
    '''
    Calculates the basic statistics needed for categorization definition
    '''
    indicator_stats = {}
    fig=plt.figure()
    for indicator in frame.indicator.unique():
        subsample = frame[frame['indicator'] == indicator]
        indicator_stats[indicator]={'min':subsample.value.min(),'max':subsample.value.max()}
        print(indicator,': ', indicator_stats[indicator]['min'],', ', indicator_stats[indicator]['max']) 
    return indicator_stats

In [66]:
"""
Categorization functions
"""
def slope(row):
    """slope one"""
    return 5*((row['value']-indicator_stats[row['indicator']]['min'])/(indicator_stats[row['indicator']]['max']-indicator_stats[row['indicator']]['min']))+1

def ws_cat(row):
    """baseline water stress"""

    if row['value'] == 0:
        return 1
    elif row['value'] < 0:
        return None
    else: 
        return max(0,min(5,((log(row['value'])-log(0.1))/(log(2)))))

def droughtsoil_cat(row):
    """baseline water stress"""
    if row['value'] < 0:
        return 0
    elif row['value'] <= 0.1:
        return 1
    elif row['value'] > 0.1 and row['value']<=0.25:
        return 2
    elif row['value'] > 0.25 and  row['value'] <=0.5:
        return 3
    elif row['value'] > 0.5 and row['value']<=0.75:
        return 4
    elif row['value'] > 0.75:
        return 5

    
def gws_cat(row):
    """Ground water stress"""
    if row['value'] == -9999:
        return None
    elif row['value'] < 1 and row['value'] >= 0 :
        return 1
    elif row['value'] < 5 and row['value'] >= 1 :
        return 2
    elif row['value'] < 10 and row['value'] >= 5 :
        return 3
    elif row['value'] < 20 and row['value'] >= 10 :
        return 4
    elif row['value'] < 9999 and row['value'] >= 20 :
        return 5
def gwt_cat(row):
    """Ground water decline trend"""
    if row['value'] == -9999:
        return None
    elif row['value'] < 0 :
        return 1
    elif row['value'] < 2 and row['value'] >= 0 :
        return 2
    elif row['value'] < 5 and row['value'] >= 2 :
        return 3
    elif row['value'] < 8 and row['value'] >= 5 :
        return 4
    elif row['value'] >= 8 :
        return 5
    

In [67]:
def categorize(row):
    '''
    Categorize the raw data on 0-5 range
    '''
    if row['basinid'] in arid_list:
        return -1
    elif row['basinid'] in stresed_list:
        return 5
    elif row['indicator']=='groundwater_stress':
        return gws_cat(row)
    elif row['indicator']=='groundwater_decline_trend':
        return gwt_cat(row)
    elif row['indicator']=='water_stess':
        return ws_cat(row)
    elif row['indicator']=='drought_severity_soil':
        return droughtsoil_cat(row)
    else:
        return slope(row)
    
    

In [68]:
indicator_stats = indicatorStc(f)
f['cat'] = f.apply(categorize, axis=1)

drought_severity_soil :  0.0 ,  5.16510901732
drought_severity_flow :  0.0 ,  53.4934771611
interannual_variability :  0.0 ,  7.0371393057
seasonal_variability :  0.0 ,  11.9822691404
water_stess :  -3.67572715908e+25 ,  inf
groundwater_stress :  -9999.0 ,  293.02
groundwater_decline_trend :  -9999.0 ,  44.0978


In [53]:
f.head(2)

,aqid,basinid,data_type,indicator,period,period_value,scenario,value,year,cat
0,None,111011,absolute,drought_severity_soil,year,None,historic,0.442394,2014,3.0
1,None,111012,absolute,drought_severity_soil,year,None,historic,0.224826,2014,-1.0


In [69]:
for indicator in f.indicator.unique():
    subsample = f[f['indicator'] == indicator]
    print(indicator,': ', subsample.cat.min(),', ', subsample.cat.max())

drought_severity_soil :  -1.0 ,  5.0
drought_severity_flow :  -1.0 ,  5.0
interannual_variability :  -1.0 ,  5.70974723561
seasonal_variability :  -1.0 ,  5.65672640592
water_stess :  -1.0 ,  5.0
groundwater_stress :  1.0 ,  5.0
groundwater_decline_trend :  1.0 ,  5.0


In [70]:
f.to_csv(base_path+'/work/data/aqueduct/dst/aqueduct/v3/water_risk_indicators_v3.csv')

## Raster reclassification

For those indicators that are going to be shown at pixel level on the visualization we need to agreed on a name convention so we can call them independenly and reclasify them following the same categorization formulas used for the basin level data.

* **Drought severity:** $$x = 5\frac{{[rawValue]}}{1} $$
* **Interannual variability:** $$x = 5\frac{{[rawValue]-min}}{max-min} + 1$$
* **Seasonal variability:** $$x = 5\frac{{[rawValue]-min}}{max-min} + 1$$

In [28]:
def droughtsoil_rcat(x):
    """baseline water stress"""
    if x < 0:
        return 0
    elif x <= 0.1:
        return 1
    elif x > 0.1 and x<=0.25:
        return 2
    elif x > 0.25 and  x <=0.5:
        return 3
    elif x > 0.5 and x<=0.75:
        return 4
    elif x > 0.75:
        return 5
    
def rCat(x):
    if data['indicator']=='drought_severity_soil':
        if x < 0:
            return 0
        elif x <= 0.1:
            return 1
        elif x > 0.1 and x<=0.25:
            return 2
        elif x > 0.25 and  x <=0.5:
            return 3
        elif x > 0.5 and x<=0.75:
            return 4
        elif x > 0.75:
            return 5
    elif x < 0:
        return 0
    else:
        return floor(5*((x-data['min'])/(data['max']-data['min'])) + 1)



In [29]:
tifStats = []
for data in tifss:
    with rasterio.open(baseLocation + data['dataset']) as tif:
        dataset = tif.read(1, masked=True)
        data['min'] = numpy.amin(dataset)
        data['max'] = numpy.amax(dataset)
        tifStats.append(data)
    
print(tifStats)    
asdf={x:{'min':[],'max':[]} for x in numpy.unique([x['indicator'] for x in tifss])} 

for data in tifStats:
    asdf[data['indicator']]['min'].append(data['min'])
    asdf[data['indicator']]['max'].append(data['max'])
asdf
tifStats

[{'dataset': 'global_interannualvariabilitywatersupply_5min_1960-2014.tif', 'indicator': 'interannual_variability', 'period': 'year', 'period_value': None, 'year': 2014, 'data_type': 'absolute', 'scenario': 'historic', 'min': 0.0, 'max': 7.6268301}]


[{'data_type': 'absolute',
  'dataset': 'global_interannualvariabilitywatersupply_5min_1960-2014.tif',
  'indicator': 'interannual_variability',
  'max': 7.6268301,
  'min': 0.0,
  'period': 'year',
  'period_value': None,
  'scenario': 'historic',
  'year': 2014}]

In [30]:
rCat = numpy.vectorize(rCat)
for data in tifStats:
    with rasterio.open(baseLocation + data['dataset']) as tif:
        print(data['dataset'])
        dataset = tif.read(1)
        f = numpy.vectorize(rCat)
        result_dataset = f(dataset)
        profile = tif.profile
        profile['dtype']=rasterio.uint8
        profile['compress']='lzw'
        profile['nodata']=0
        fileName = '{0}-{1}-{2}-{3}{4}.tif'.format(data['indicator'],data['scenario'],data['period'],str(data['year']),('-'+str(data['period_value'])) if data['period_value']!= None else '')
        filedir=base_path + '/work/data/aqueduct/dst/aqueduct/v3/' +  fileName
        with rasterio.open(filedir, 'w', **profile) as dst:
            dst.write(result_dataset.astype(profile['dtype']), 1)


global_interannualvariabilitywatersupply_5min_1960-2014.tif


## Rankings

In [16]:
xlsfiles = []
for dirpath, subdirs, files in os.walk(base_path+'/work/data/aqueduct/data_source/aqueduct/v3/rankings'):
    for x in files:
        if x.endswith(".xls"):
            info = x.split('.')[0].split('-')
            xlsfiles.append({"path":os.path.join(dirpath, x),"crop":info[1], "indicator":info[0][:2], "year":(2000+int(info[0][2:4])), "irrigation":info[2]})

In [18]:
dict_ind={"ws":"water_stress",
          "ds":"drought_stress"}
dataframes = []
for file in xlsfiles:
    data = pd.read_excel(file["path"])
    data['crop']= file["crop"]
    data['year']= file["year"]
    data['irrigation']= file["irrigation"]
    data['value']= data['AGGR_VALUES']
    data['indicator']= dict_ind[file["indicator"]]
    dataframes.append(data)
d=pd.concat(dataframes)
d[['crop','irrigation','year','value','indicator','ISO', 'NAME_ISO', 'POP2000', 'SQKM']].to_csv(base_path+'/work/data/aqueduct/dst/aqueduct/v3/water_risk_rankings_v3.csv')

## Data Migration from wri-01 account to wri-rw

Carto has an issue with big tables > 1000000 rows tables. In order to ingest
### todolist
- [x] split crops, and crops_geo table
- [x] upload it to carto

In [ ]:
import requests
import json

In [ ]:
url = 'https://wri-rw.carto.com/api/v1/viz/?tag_name=aqueduct&type=&exclude_shared=false&per_page=1000&tags=aqueduct&types=table'
r = requests.get(url)
tableStructure= pd.read_json(json.dumps(r.json()['visualizations']), orient='records')
tableStructure.head(17)

In [ ]:
"""
https://carto.com/docs/carto-engine/import-api/quickstart
"""
import_url= "https://wri-rw.carto.com/api/v1/imports/"
payload = {
    "api_key":""
}
def split_table(dataset_url, name):
    """
    this function splits a table in x datasets to make it uploable to a carto account
    """
    size=1000000
    dataset_data=[]
    format_file = dataset_url.split('.')[1]
    geometry_formats=['shp']
    nongeo_formats=['csv']
    print('---------------------')
    if format_file in nongeo_formats:
        dataset = pd.read_file(dataset_url)
        rows, columns = dataset.shape
        ra=list(range(0, rows+1, size))
        if (rows%size)>0:
            ra.append(rows+1)
        print('---------------------')
        for i, data in enumerate(ra):
            if i<(len(ra)-1):
                split_dataset_name = 'split'+'_'+name+'_'+str(i)+'.csv'
                full_d_path = base_path+'/work/data/aqueduct/dst/aqueduct/v3/'+split_dataset_name
                dataset.iloc[:,ra[i]:ra[i+1]].to_csv(full_d_path)
                dataset_data.append({"part_name":split_dataset_name,"path":full_d_path})
        return {"table_name":name,"parts":dataset_data}
    elif format_file in geometry_formats:
        dataset = gpd.read_file(dataset_url)
        rows, columns = dataset.shape
        ra=list(range(0, rows+1, size))
        if (rows%size)>0:
            ra.append(rows+1)
        print('---------------------')
        for i, data in enumerate(ra):
            if i<(len(ra)-1):
                split_dataset_name = 'split'+'_'+name+'_'+str(i)+'.shp'
                full_d_path = base_path+'/work/data/aqueduct/dst/aqueduct/v3/'+split_dataset_name
                dataset.iloc[:,ra[i]:ra[i+1]].to_file(full_d_path)
                dataset_data.append({"part_name":split_dataset_name,"path":full_d_path})
        return {"table_name":name,"parts":dataset_data}
    else:
        raise ValueError
        print('format not supported')
    
    
    
def upload_carto(parts):
    """
    this function will upload a dataset into carto
    """
    dict_response = []
    for x in parts:
        files = {'file': (x['part_name'], open(x['path'], 'rb'), 'application/zip', {'Expires': '0'})}
        r = requests.post(url, params=payload, files=files) 
        response = { **x, **r.json()}
        dict_response.append(response)
    return dict_response
    
    
def merge_carto(parts):
    """
    this function will merge datasets into carto
    """
    meta_sql = "SELECT * INTO job_result FROM ({{query}}) AS job"
    sql="select {{columns}} from {{datasets}}"
    union = "UNION ALL"

def daterwwer(file, name):
    d = split_table(file, name)
    #t = upload_carto(d['parts'])

In [ ]:
daterwwer(base_path + '/work/data/aqueduct/data_source/spamdata/crops/crops.shp', 'crops')

In [ ]:
dataset_url=base_path + '/work/data/aqueduct/data_source/spamdata/crops/crops.shp'

dataset = gpd.read_file(dataset_url)

In [ ]:
rows, columns = dataset.shape


print(rows)

In [ ]:
rows, columns = dataset.shape
ra=list(range(0, rows+1, size))
if (rows%size)>0:
    ra.append(rows+1)
print('---------------------')
for i, data in enumerate(ra):
    if i<(len(ra)-1):
        split_dataset_name = 'split'+'_crops_'+str(i)+'.shp'
        full_d_path = base_path+'/work/data/aqueduct/dst/aqueduct/v3/'+split_dataset_name
        dataset.iloc[:,ra[i]:ra[i+1]].to_file(full_d_path)
        dataset_data.append({"part_name":split_dataset_name,"path":full_d_path})

In [ ]:
os.getcwd()